In [5]:
# let's take one dataset for example
from multilab.datasets import reuter
sentences, labels = reuter()

[nltk_data] Downloading package reuters to /Users/monk/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [6]:
# very few datapoints just to demonstrate how to use models

# preprocessing
from multilab.preprocess import Text_preprocessing
text_preprocessing = Text_preprocessing()
dataframe = text_preprocessing.labels_to_dataframe(sentences,labels)
preprocessded_dataset = text_preprocessing.initial_preprocess(dataframe, chunk_value = 5)
dataset_s , frequency_list_s = text_preprocessing.keep_labels(preprocessded_dataset,
                                                           keep_ratio=0.1)

# text preprocessing
slice_dataset = text_preprocessing.dataset_slice(dataset_s,ratio=0.01)

#tf-idf
all_sentences, all_labels = text_preprocessing.tf_idf(slice_dataset)

# split the dataset
X_train, X_test, y_train, y_test = text_preprocessing.split_dataset(all_sentences, all_labels)

lower_case done
punctuation removed
text cleaning done


In [8]:
from multilab.models import BinaryRe
Bm = BinaryRe(X_train, y_train, X_test,y_test)
print(Bm.train())

# four base models

# BinaryRelevance
# ClassifierChain
# LabelPowerset
# Mlknn


{'accuracy': 0.4444444444444444, 'f1_score': 0.38383838383838387}


In [7]:
from multilab.models import Classfierchains
Cc = Classfierchains(X_train, y_train, X_test,y_test)
print(Cc.train())

{'accuracy': 0.5185185185185185, 'f1_score': 0.5357142857142857}


/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 

In [9]:
from multilab.models import labelpowerset
lp = labelpowerset(X_train, y_train, X_test,y_test)
print(lp.train())

{'accuracy': 0.5185185185185185, 'f1_score': 0.5357142857142857}


/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [13]:
import configparser
import pickle as pk
import tensorflow as tf
from tqdm import tqdm
from tqdm import trange

from hm import hamming_score
from sklearn.metrics import f1_score
from Bilstm_simples import Base_model


import random
import time
import os 
import numpy as np
import pickle as pk
from sklearn.model_selection import train_test_split

#load configuation

config = configparser.RawConfigParser()
config.read('config.properties')
parameter_dict = dict(config.items('BiLstm network'))
boolean_dict = {'None': None, 'True': True, 'False': False}

#load data files 
with open(parameter_dict['sentence_path'] ,'rb') as f:
    X_data = pk.load(f)

with open(parameter_dict['labels_path']   ,'rb') as f:
    y_data = pk.load(f)

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, train_size=0.8)




def get_train_data(batch_size, slice_no):


    batch_data_j = np.array(X_train[slice_no * batch_size:(slice_no + 1) * batch_size])
    batch_labels = np.array(y_train[slice_no * batch_size:(slice_no + 1) * batch_size])
    
    max_sequence = max(list(map(len, batch_data_j)))

    # getting Max length of sequence
    padded_sequence = [i + [0] * (max_sequence - len(i)) if len(i) < max_sequence else i for i in batch_data_j]
    
    return {'sentenc': padded_sequence, 'labels': batch_labels}


def get_test_data(batch_size,slice_no):


    batch_data_j = np.array(X_val[slice_no * batch_size:(slice_no + 1) * batch_size])
    batch_labels = np.array(y_val[slice_no * batch_size:(slice_no + 1) * batch_size])
    
    max_sequence = max(list(map(len, batch_data_j)))
    
    padded_sequence = [i + [0] * (max_sequence - len(i)) if len(i) < max_sequence else i for i in batch_data_j]
    
    return {'sentenc': padded_sequence, 'labels': batch_labels}



def evaluate_(model, epoch_, batch_size = 120):

    sess = tf.get_default_session()
    iteration = len(X_val) // batch_size

    sub_accuracy    = []
    hamming_score_a = []
    hamming_loss_   = []

    micr_ac = []
    weight_ac = []

    for i in range(iteration):
        
        data_g = get_test_data(batch_size,i)
        
        sentences_data = data_g['sentenc']
        labels_data    = data_g['labels']

        network_out, targe = sess.run([model.predictions,model.targets], feed_dict={model.placeholders['sentence']: sentences_data,
                                                                                    model.placeholders['labels']: labels_data, 
                                                                                    model.placeholders['dropout']: 0.0})

        h_s     = hamming_score(targe, network_out)

        ham_sco = h_s['hamming_score']
        sub_acc = h_s['subset_accuracy']
        ham_los = h_s['hamming_loss']

        sub_accuracy.append(sub_acc)
        hamming_score_a.append(ham_sco)
        hamming_loss_.append(ham_los)



        micr_ac.append(f1_score(targe, network_out, average='micro'))
        weight_ac.append(f1_score(targe, network_out, average='weighted'))

    return {  'subset_accuracy' : np.mean(np.array(sub_accuracy)) , 
              'hamming_score'   : np.mean(np.array(hamming_score_a)) , 
              'hamming_loss'    : np.mean(np.array(hamming_loss_)), 
               'micro_ac'       : np.mean(np.array(micr_ac)), 
               'weight_ac'      : np.mean(np.array(weight_ac)) , 'epoch': epoch_ }





def train_model(model, batch_size = int(parameter_dict['batch_size']), epoch = int(parameter_dict['epoch'])):
    with tf.Session() as sess:
        sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
        iteration = len(X_train) // batch_size


        for i in range(epoch):
            t = trange(iteration, desc='Bar desc', leave=True)

            for j in t:



                data_g = get_train_data(batch_size,j)
                sentences_data = data_g['sentenc']
                labels_data    = data_g['labels']



                network_out, train, targe, losss  = sess.run([model.predictions, model.optimizer, model.targets,model.loss],
                                          feed_dict={model.placeholders['sentence']: sentences_data,
                                                     model.placeholders['labels']: labels_data,
                                                     model.placeholders['dropout']: 0.2})

                t.set_description("epoch {},  iteration {},  F1_score {},  loss {}".format(i,
                                                                                       j,
                                                                                       f1_score(targe, 
                                                                                                network_out, 
                                                                                                average='micro'), 
                                                                                       losss))
                t.refresh() # to show immediately the update


            val_data = evaluate_(model, i, batch_size = 100)
            print("validation_acc",val_data)
            with open('./result/iterres.txt', 'a') as f:
                f.write(str({'test_accuracy':  val_data}) + '\n')
                
                
if __name__ == "__main__":
    
    model = Base_model(vocab_size                  =   int(parameter_dict['vocab_size']),
                       rnn_units                   =   int(parameter_dict['rnn_units']), 
                       word_embedding_dim          =   int(parameter_dict['word_embedding_dim']),  
                       no_of_labels                =   int(parameter_dict['no_of_labels']), 
                       learning_rate               =   float(parameter_dict['learning_rate']),   
                       trained_embedding           =   boolean_dict[parameter_dict['trained_embedding']], 
                       train_embedding             =   boolean_dict[parameter_dict['train_embedding']],
                       model_output                =   boolean_dict[parameter_dict['model_output']])
    
    train_model(model)

ModuleNotFoundError: No module named 'hm'

In [14]:
import tensorflow as tf
import numpy as np




class Bilstm(object):
    
    def __init__(self, X_train, y_train, X_test, y_test, configuration = None):
        
        self.X_train = X_train
        self.y_train = y_train
        self.X_test  = X_test
        self.y_test  = y_test
        
        self.old_configuration = {
                         'vocab_size'                 : 2000, 
                         'rnn_units'                  : 256, 
                         'word_embedding_dim'         : 300, 
                         'learning_rate'              : 0.001, 
                         'pretrained_embedding_matrix': None,
                         'dropout'                    : 0.2,
                         'epoch'                      : 1,
                         'batch_size'                 : 128,
                         'result_path'                : '/Users/monk/Desktop/'
                        }
        
        if configuration:
            self.old_configuration.update(configuration)
            
        if self.old_configuration['pretrained_embedding_matrix'] is None:
            self.old_configuration['pretrained_embedding_matrix'] = None
        else:
            with open(self.old_configuration['pretrained_embedding_matrix'], 'rb') as f:
                self.embedding_mat = np.array(pk.load(f))
                
            
        
    
    def default_configuration(self):
        
        default_conf = {
                        'vocab_size'      : 'vocab_size of corpus', 
                        'rnn_unit'        : 'bi-directional_rnn units', 
                        'embedding_dim'   : 'word_embedding_dim', 
                        'learning rate'   : 'learning rate of model', 
                        'embedding_matrix': 'path of embedded matrix  (ex glove, elmo )',
                        'train_embedding' : 'train glove embedding or not', 
                        'last_output'     : 'use lstm last state or use final output of all states'
                       }
        return default_conf
    
    
    # train data loader
    def get_train_data(self, batch_size, slice_no):


        batch_data_j = np.array(X_train[slice_no * batch_size:(slice_no + 1) * batch_size])
        batch_labels = np.array(y_train[slice_no * batch_size:(slice_no + 1) * batch_size])

        max_sequence = max(list(map(len, batch_data_j)))

        # getting Max length of sequence
        padded_sequence = [i + [0] * (max_sequence - len(i)) if len(i) < max_sequence else i for i in batch_data_j]

        return {'sentenc': padded_sequence, 'labels': batch_labels }
    
    
    # test data loader
    def get_test_data(self, batch_size,slice_no):


        batch_data_j = np.array(X_val[slice_no * batch_size:(slice_no + 1) * batch_size])
        batch_labels = np.array(y_val[slice_no * batch_size:(slice_no + 1) * batch_size])

        max_sequence = max(list(map(len, batch_data_j)))

        padded_sequence = [i + [0] * (max_sequence - len(i)) if len(i) < max_sequence else i for i in batch_data_j]

        return {'sentenc': padded_sequence, 'labels': batch_labels}
    
    
    def evaluate_(self, model, epoch_, batch_size = 120):

        sess = tf.get_default_session()
        iteration = len(X_val) // batch_size

        sub_accuracy    = []
        hamming_score_a = []
        hamming_loss_   = []

        micr_ac = []
        weight_ac = []

        for i in range(iteration):

            data_g = self.get_test_data(batch_size,i)

            sentences_data = data_g['sentenc']
            labels_data    = data_g['labels']

            network_out, targe = sess.run([model.predictions,model.targets], feed_dict={model.placeholders['sentence']: sentences_data,
                                                                                        model.placeholders['labels']: labels_data, 
                                                                                        model.placeholders['dropout']: 0.0})

            h_s     = hamming_score(targe, network_out)

            ham_sco = h_s['hamming_score']
            sub_acc = h_s['subset_accuracy']
            ham_los = h_s['hamming_loss']

            sub_accuracy.append(sub_acc)
            hamming_score_a.append(ham_sco)
            hamming_loss_.append(ham_los)



            micr_ac.append(f1_score(targe, network_out, average='micro'))
            weight_ac.append(f1_score(targe, network_out, average='weighted'))

        return {  'subset_accuracy' : np.mean(np.array(sub_accuracy)) , 
                  'hamming_score'   : np.mean(np.array(hamming_score_a)) , 
                  'hamming_loss'    : np.mean(np.array(hamming_loss_)), 
                   'micro_ac'       : np.mean(np.array(micr_ac)), 
                   'weight_ac'      : np.mean(np.array(weight_ac)) , 'epoch': epoch_ }
    
    
    
    
    def train_model(model, batch_size = int(self.old_configuration['batch_size']), epoch = int(self.old_configuration['epoch'])):
        with tf.Session() as sess:
            sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
            iteration = len(X_train) // batch_size


            for i in range(epoch):
                t = trange(iteration, desc='Bar desc', leave=True)

                for j in t:



                    data_g = get_train_data(batch_size,j)
                    sentences_data = data_g['sentenc']
                    labels_data    = data_g['labels']



                    network_out, train, targe, losss  = sess.run([model.predictions, model.optimizer, model.targets,model.loss],
                                              feed_dict={model.placeholders['sentence']: sentences_data,
                                                         model.placeholders['labels']: labels_data,
                                                         model.placeholders['dropout']: 0.2})

                    t.set_description("epoch {},  iteration {},  F1_score {},  loss {}".format(i,
                                                                                           j,
                                                                                           f1_score(targe, 
                                                                                                    network_out, 
                                                                                                    average='micro'), 
                                                                                           losss))
                    t.refresh() # to show immediately the update


                val_data = evaluate_(model, i, batch_size = 100)
                print("validation_acc",val_data)
                with open(self.old_configuration['result_path'], 'a') as f:
                    f.write(str({'test_accuracy':  val_data}) + '\n')
                    
        
                    
    
    
    
    
    
    
    

In [29]:
a = {'a':1,'b':2,'c':3}

npo = np.random.uniform(0,10,[12,34])
cus = None

if npo is None:
    npo = None
else:
    print(npo)
    
    

[[6.96749427 8.37410883 8.93786136 4.858598   4.23367702 3.90523282
  6.90688789 6.73677213 9.21940306 9.6061811  1.76435984 3.11601394
  3.609397   6.99067604 8.11557778 3.78312029 1.32424771 6.99604939
  6.26806372 8.84764374 1.52276249 4.37953128 8.96641546 5.82352097
  4.24953579 1.76626568 6.74539449 4.05552095 1.32276495 5.84345404
  9.47571667 9.09332491 9.66537468 5.31360579]
 [5.0965566  2.89707967 4.42813393 6.47907003 8.26568389 9.35974264
  5.94795408 9.57289163 8.22587112 3.48316327 6.74307239 2.03558089
  2.6252177  6.97514451 6.11062897 1.05484058 4.01252538 1.88531099
  7.58288896 8.33064739 1.30870082 0.23070554 8.05773683 1.63856073
  2.50701087 4.33130697 6.34362772 5.82573735 7.27797961 7.86481944
  8.75459193 8.58835285 7.81609895 8.77164648]
 [0.75212823 9.99761552 9.62505458 3.35114296 1.2840918  1.82157585
  8.77775895 8.56732858 8.02259777 0.33052952 7.55428328 0.92367731
  4.67996352 9.16923802 1.05494592 0.90791769 7.62663702 3.88363306
  7.44629013 7.9900496